# Capstone Week 3
## Q1

In [1]:
#conda install -c anaconda lxml --yes
#! pip3 install lxml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import folium


In [2]:
#req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
can_html = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
can_df = can_html[0]
can_df = can_df[can_df["Borough"]!="Not assigned"]
can_df.reset_index(drop=True, inplace = True)
can_df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [3]:
print(can_df.shape)

(103, 3)


 # Q2
 ## Here I did use arcgis isntead of google, since I could do more requests per day and since I did wrong on my first request on google, so with arcgis I could get more tries.

In [4]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder
from geopy.geocoders import Nominatim

In [5]:
latitude=[]
longitude=[]
for code in can_df['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

In [6]:
can_df["Latitude"] = latitude
can_df["Longitude"] = longitude
can_df.head(12)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


# Q3

In [7]:
address = "Toronto, Canada"
geolocator = Nominatim(user_agent = "can_explorer")
location = geolocator.geocode(address)
latitude =  location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


 Creating the map of Toronto and then create the Marks on the neighborhood of Toronto

In [8]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)
for latitude, longitude, neigh, borough in zip(can_df["Latitude"], can_df["Longitude"], can_df.Neighbourhood, can_df.Borough):
    label = "{}, {}".format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        color = "red",
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
map_toronto

## Creating KMean model and scale the latitud and longtitud to a faster computation on the model
## Also I decide to use 4 Clusters to divide on East, West, North and South Zones

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
Kmeans_model = KMeans(n_clusters = 4, init="k-means++", n_init = 12)
X = can_df[["Latitude", "Longitude"]]
X = StandardScaler().fit_transform(X)
Kmeans_model.fit(X)
labels_cluster = Kmeans_model.labels_
can_df["Zone"] = labels_cluster
can_df.head(5)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Zone
0,M3A,North York,Parkwoods,43.75245,-79.32991,3
1,M4A,North York,Victoria Village,43.73057,-79.31306,3
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,2
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,1
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,2


In [10]:
for latitude, longitude, neigh, borough, zone in zip(can_df["Latitude"], can_df["Longitude"], can_df.Neighbourhood, can_df.Borough, can_df.Zone):
     if (zone==0):
        label = "{}, {}".format(neigh, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [latitude, longitude],
            radius = 5,
            color = "red",
            fill = True,
            fill_color = 'red',
            fill_opacity = 0.7,
            parse_html = False).add_to(map_toronto)
     elif (zone==1):
        label = "{}, {}".format(neigh, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [latitude, longitude],
            radius = 5,
            color = "blue",
            fill = True,
            fill_color = 'blue',
            fill_opacity = 0.7,
            parse_html = False).add_to(map_toronto)
     elif (zone==2):
        label = "{}, {}".format(neigh, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [latitude, longitude],
            radius = 5,
            color = "green",
            fill = True,
            fill_color = 'green',
            fill_opacity = 0.7,
            parse_html = False).add_to(map_toronto)
     else :
        label = "{}, {}".format(neigh, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [latitude, longitude],
            radius = 5,
            color = "yellow",
            fill = True,
            fill_color = 'yellow',
            fill_opacity = 0.7,
            parse_html = False).add_to(map_toronto)
        
map_toronto